<a href="https://colab.research.google.com/github/SOMYAKAHAR/generative-ai/blob/main/M2_Assignment_Data_Analysis_with_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 2 Coding Assignment: Retail Sales Data Analysis with LLM


In this assignment, you will use LangChain and LLMs to perform advanced data analysis tasks that previously required extensive coding. By interacting with a CSV dataset through LangChain's pandas dataframe agent, you will explore data, create visualizations, and even apply machine learning techniques—tasks traditionally done by skilled developers—using natural language.

# Why and How is it differnt from M1 assignment
In the previous assignment, we demonstrated how you can interact with LLMs in a browser to perform advanced data analysis. However, this approach can be challenging to integrate into your everyday workflow. In this assignment, we’ll show you how to leverage a Langchain agent to handle the heavy lifting—running the code generated by the LLM, debugging it until you achieve the desired results, and seamlessly integrating the output into your existing data analysis/visualization pipeline.

# Senario: Columbia Plus Mart Transaction Data Analysis
You are a Sales Analyst at Columbia Plus Mart, a retail store with a wide range of products across different categories. Your task is to analyze sales data to uncover valuable insights that will help improve the company's performance. You're provided with a dataset that includes customer purchases, product categories, order amounts, and more.

# Tasks
## 1. Data Exploration

Use the Langchain agent with pandas to explore and clean the dataset. Identify any missing values, outliers, or trends.

## 2. Sales Analysis

Explore when order hikes or drops occur (e.g., specific months, days of the week, or during sales events).

## 3. Customer Insights with Machine Learning

Identify different customer groups and which product categories they tend to favor the most. This might involve clustering based on spending patterns, demographics, or product categories.

## 4. Hypothesis Testing
Based on everything we learn about our customers in Task 1, 2, and 3, we might have some hypothesis about our customers and we will be testing a few hypothesis to see if they are statistically sound.

# More on the dataset
You can download the dataset here: https://www.kaggle.com/datasets/manjeetsingh/retaildataset

The dataset contains the following columns:

* Transaction ID: Unique identifier for each transaction.

* Date: The date of the transaction.

* Customer ID: Unique identifier for each customer.

* Gender: The gender of the customer.

* Age: The age of the customer.

* Product Category: The category of the product purchased (e.g., Beauty, Clothing, Electronics).

* Quantity: The number of units purchased.

* Price per Unit: The price of a single unit of the product.

* Total Amount: The total cost of the transaction (Quantity * Price per Unit).




# Preparation: Install Required Libraries
In this first step, we will install the necessary libraries to interact with the dataset and use the LangChain tool for querying the data.


In [2]:
# freezing the coding environment to these specific versions
!pip install openai==1.72.0 langchain==0.3.23 langchain_openai==0.3.13 langchain_experimental==0.3.4 langchain_community==0.3.21 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.9/643.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 92.0 MB/s eta 0:00:00


# Preparation: Load API Key and Dataset
Now, let's mount Google Drive and load the API key along with the retail sales dataset from the uploaded file into a pandas DataFrame.

***If you don't have an OpenAI API key yet, please visit https://platform.openai.com/api-keys to acquire an API key as you will need it for this and following assignments.***

Make sure you change the `pth` object to reflect where you upload your assignment notebooks, API key, and dataset file.

In [3]:
from langchain_openai import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent
import openai
import pandas as pd
import os

# Mount Google Drive if using Colab
# try:
from google.colab import drive
drive.mount('/content/drive')
pth = '/content/drive/MyDrive/Colab_Notebooks'

# Change working directory
os.chdir(pth)

# Read the key from file and set environment variable
with open("api/santoshOpenai.txt", "r") as f:
    key = f.read().strip()
os.environ["OPENAI_API_KEY"] = key   # ✅ Correct env var name

# Path to the dataset (since you're already inside Colab_Notebook)
file_path = "reconstructed_merged_full.csv"

# Load dataset
df = pd.read_csv(file_path)

# Show preview
print(df.head(10))

print("Using key:", os.environ["OPENAI_API_KEY"])



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   Store  Dept        Date  Weekly_Sales  IsHoliday Type    Size  Temperature  \
0      1     1  05/02/2010      24924.50      False    A  151315        42.31   
1      1     1  12/02/2010      46039.49       True    A  151315        38.51   
2      1     1  19/02/2010      41595.55      False    A  151315        39.93   
3      1     1  26/02/2010      19403.54      False    A  151315        46.63   
4      1     1  05/03/2010      21827.90      False    A  151315        46.50   
5      1     1  12/03/2010      21043.39      False    A  151315        57.79   
6      1     1  19/03/2010      22136.64      False    A  151315        54.58   
7      1     1  26/03/2010      26229.21      False    A  151315        51.45   
8      1     1  02/04/2010      57258.43      False    A  151315        62.27   
9      1     1  09/04/2010      42960.91      False    A  151

In [4]:
!pip install -q -U google-genai
!pip install -q -U langchain-google-genai google-genai
!pip install langchain langchain-openai langchain-experimental
!pip install langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.6/245.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [6]:
# !pip install -q -U google-genai

# import pandas as pd
# import os
# from google.colab import drive
# import google.genai as genai

# # --- Mount Google Drive ---
# drive.mount('/content/drive')
# pth = '/content/drive/MyDrive/Colab_Notebooks'
# os.chdir(pth)

# # --- Load Gemini API key ---
# with open("api/gemini.txt", "r") as f:   # store your Gemini API key in this file
#     GEMINI_API_KEY = f.read().strip()

# # --- Create Gemini Client ---
# client = genai.Client(api_key=GEMINI_API_KEY)

# # --- Load Dataset ---
# file_path = "reconstructed_merged_full.csv"
# df = pd.read_csv(file_path)
# print("✅ Data Loaded:", df.shape)
# df.head(10)


# Preparation: Create LangChain Agent

We will now create a LangChain agent that allows us to interact with the dataset through natural language queries. This agent will analyze the data and respond with insights.



In [7]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2
)

# set up the pandas dataframe agent
agent_executor = create_pandas_dataframe_agent(llm,
                                      df,
                                      number_of_head_rows=429761, # Set this to the total number of rows in df
                                      agent_type="openai-tools",
                                      allow_dangerous_code = True,
                                      verbose=True)

# Now we got all the setup taken care of, let's get start with our tasks!

# Prompt Engineering Tip
After some experimentation, I found that it works best when you attach

***The dataset is available at data/retail_sales_dataset.csv, and I’d like you to use all 1000 rows. Thank you!***

at the end of your prompt. Keep this in mind when you perform the below tasks.


# Step 1: Data Exploration

Use the Langchain agent with pandas to explore and clean the dataset. Identify any missing values, outliers, or trends.

## Sample prompt
***Can you help me explore and clean the dataset. Identify any missing values, outliers, or trends? The data you need is in data/retail_sales_dataset.csv. The dataset is available at data/retail_sales_dataset.csv, and I’d like you to use all 1000 rows. Thank you!***


In [8]:
# Load your dataset
df = pd.read_csv("merged_union.csv")

# ---- Missing values analysis ----
missing_summary = df.isnull().sum()
missing_summary = missing_summary[missing_summary > 0]  # only show columns with missing values

# ---- Outlier detection (using IQR) ----
outliers_dict = {}
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
    if not outliers.empty:
        outliers_dict[col] = len(outliers)

outliers_df = pd.DataFrame(list(outliers_dict.items()), columns=['Column', 'Outlier_Count'])

# ---- Show results ----
print("🔎 Missing Values Summary:\n", missing_summary)
print("\n📊 Outliers Summary:\n", outliers_df)

🔎 Missing Values Summary:
 Temperature     421570
Fuel_Price      421570
MarkDown1       425728
MarkDown2       426839
MarkDown3       426147
MarkDown4       426296
MarkDown5       425710
CPI             422155
Unemployment    422155
Dept              8190
Weekly_Sales      8190
Type            429760
Size            429760
dtype: int64

📊 Outliers Summary:
          Column  Outlier_Count
0   Temperature              7
1     MarkDown1            237
2     MarkDown2            436
3     MarkDown3            480
4     MarkDown4            337
5     MarkDown5            212
6  Unemployment            386
7  Weekly_Sales          35521


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)
resp = llm.invoke("Hello Gemini, say hi in one short sentence.")
print(resp.content)


Streaming output truncated to the last 5000 lines.
  File "/usr/local/lib/python3.12/dist-packages/google/auth/credentials.py", line 239, in before_request
    self._blocking_refresh(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/credentials.py", line 202, in _blocking_refresh
    self.refresh(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 132, in refresh
    raise new_exc from caught_exc
google.auth.exceptions.RefreshError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7db864b15fa0>)
ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7db869317da0>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist

In [10]:
# Change this prompt and observe how the response changes
prompt = """
Can you help me explore and clean the dataset.
Identify any missing values, outliers, or trends.
"""
response = agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [4]:

import os
from langchain.agents import AgentType, initialize_agent, Tool
from langchain_openai import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
# Change this prompt and observe how the response changes
prompt = """
Identify any missing values, outliers, or trends.
"""
response = agent_executor.invoke(prompt)

# # Example: create a short summary text to send to the agent
# summary_text = f"""
# I have a dataset with {df.shape[0]} rows and {df.shape[1]} columns.
# Here are the top 10 columns by missing percent:
# {missing_summary.head(10).to_dict()}
# Top outlier columns:
# {outliers_df.head(5).to_dict()}
# """
# response = agent_executor.invoke(summary_text)
# # Pass that short text to the agent (or use agent.run)
# summary_text=agent_executor.run("Please inspect the summary below and suggest cleaning steps:\n\n" + summary_text)




> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
!pip install langchain_google_genai
import os
from langchain.agents import AgentType, initialize_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.tools import PythonREPLTool

# 1. Set your OpenAI API key
# Use the OpenAI API key loaded earlier
with open("api/santoshOpenai.txt", "r") as f:
    OPENAI_API_KEY = f.read().strip()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY # Setting the environment variable for OpenAI


# 2. Initialize the OpenAI Language Model
llm = ChatOpenAI(model="gpt-4o", temperature=0.2) # Using a suitable OpenAI model

# 3. Define the Tools the agent can use
tools = [PythonREPLTool()]

# 4. Create the Agent Executor (THIS MUST COME BEFORE YOU USE IT)
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# NOW you can use agent_executor.invoke()
prompt = """
Identify any missing values, outliers, or trends.
"""
response = agent_executor.invoke({"input": prompt})
print(response['output'])

# # Example: create a short summary text to send to the agent
# summary_text = f"""
# I have a dataset with {df.shape[0]} rows and {df.shape[1]} columns.
# Here are the top 10 columns by missing percent:
# {missing_summary.head(10).to_dict()}
# Top outlier columns:
# {outliers_df.head(5).to_dict()}
# """
# response = agent_executor.invoke({"input": summary_text})
# # Pass that short text to the agent
# cleaning_suggestions = agent_executor.invoke({"input": "Please inspect the summary below and suggest cleaning steps:\n\n" + summary_text})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Let's take a look at the return object
The chain consists of both the input we provided to the LangChain agent and the agent’s output after executing the code on our behalf. The agent runs the code, processes the results, and returns the relevant insights, such as the summary statistics, missing values, and trends found in the dataset.

In [6]:
response

NameError: name 'response' is not defined

## Print it out for better readability

In [ ]:
print(response['output'])
# This is markup format, and you can copy this to a text cell for better readability

# Step 2: Sales Analysis

Explore when order hikes or drops occur (e.g., specific months, days of the week, or during sales events).

## Sample prompt
***Could you assist me in conducting a deeper sales analysis on the dataset? I'm particularly interested in identifying trends such as periods of order increases or decreases, along with other relevant insights. Please use visualizations to present the findings clearly. The data you need is in data/retail_sales_dataset.csv. Use all 1000 rows of it. Thank you!***

In [ ]:
# Change this prompt and observe how the response changes
prompt = """
Could you assist me in conducting a deeper sales analysis on the dataset?
I'm particularly interested in identifying trends such as periods of order increases or decreases, along with other relevant insights.
Please use visualizations to present the findings clearly.
The dataset is available at data/retail_sales_dataset.csv, and I’d like you to use all 1000 rows. Thank you!
"""
response = agent_executor.invoke(prompt)

## Review the agent's response and what's your comment?

In [ ]:
print(response['output'])

# Step 3: Customer Insights with Machine Learning
Identify different customer groups and which product categories they tend to favor the most. This might involve clustering based on spending patterns, demographics, or product categories.

## Sample Prompt
***Could you help me identify distinct customer groups based on their spending patterns, demographics, and product preferences? I'd also like recommendations on sales strategies for each group. Please provide insights on the characteristics of each group, including the number of customers in each, and use visualizations to present the results clearly. The dataset is available at data/retail_sales_dataset.csv, and I’d like you to use all 1000 rows. Thank you!***

In [ ]:
# Change this prompt and observe how the response changes
prompt = """
Could you help me identify distinct customer groups based on their spending patterns, demographics, and product preferences?
I'd also like recommendations on sales strategies for each group.
Please provide insights on the characteristics of each group, including the number of customers in each.
Use visualizations to present the results clearly.
The dataset is available at data/retail_sales_dataset.csv, and I’d like you to use all 1000 rows. Thank you!
"""
response = agent_executor.invoke(prompt)

## Let's review the agent's response closely

* Does the clustering make sense to you?
* How about the sales strategies for each cluster.

In [ ]:
print(response['output'])

# Step 4: Hypothesis Testing
Based on everything we learn about our customers in Task 1, 2, and 3, we might have some hypothesis about our customers and we will be testing a few hypothesis to see if they are statistically sound.



## Hypothesis 1: Customer age has no effect on quantity of purchase.

## Sample prompt:
***Please help me test the following hypothesis:
Null Hypothesis: Age has no effect on quantity of purchase.
Alternate hypothesis: Age has an effect on quantity of purchase.
The dataset is available at data/retail_sales_dataset.csv, and I’d like you to use all 1000 rows.***

In [ ]:
# Change this prompt and observe how the response changes
prompt = """
Please help me test the following hypothesis:
Null Hypothesis: Age has no effect on quantity of purchase.
Alternate hypothesis: Age has an effect on quantity of purchase.
The dataset is available at data/retail_sales_dataset.csv, and I’d like you to use all 1000 rows.
"""
response = agent_executor.invoke(prompt)

In [ ]:
print(response['output'])

## Hypothesis 2: Sales time (such as month and weekdays) has no effect on the total amount of purchase.

## Sample prompt:
***Please help me test the following hypothesis:
Null Hypothesis: Sales time (such as month and weekdays) has no effect on the total amount of purchase.
Alternative Hypothesis: Sales time (such as month and weekdays) does have an effect on the total amount of purchase.
The dataset is available at data/retail_sales_dataset.csv, and I’d like you to use all 1000 rows.***

In [ ]:
# Change this prompt and observe how the response changes
prompt = """
Please help me test the following hypothesis:
Null Hypothesis: Sales time (such as month and weekdays) has no effect on the total amount of purchase.
Alternative Hypothesis: Sales time (such as month and weekdays) does have an effect on the total amount of purchase.
The dataset is available at data/retail_sales_dataset.csv, and I’d like you to use all 1000 rows.
"""
response = agent_executor.invoke(prompt)


In [ ]:
print(response['output'])

## What other hypothesis do you have? Test it out here!

# Summary

In this notebook, we leveraged the power of LangChain and LLMs to perform advanced retail data analysis tasks. By interacting with retail sales dataset through a LangChain pandas DataFrame agent, we were able to:

* **Explore and Clean the Data:** We identified missing values, outliers, and initial trends in the dataset, ensuring data quality for further analysis.
* **Analyze Sales Trends:** We uncovered insights into sales patterns, including periods of order increases and decreases, providing valuable information for strategic decision-making.
* **Gain Customer Insights:** We identified distinct customer groups based on spending habits, demographics, and product preferences. These insights helped in understanding customer behavior and tailoring sales strategies.
* **Test Hypotheses:** We tested statistically sound hypotheses about customer behavior, such as the relationship between age and purchase quantity and the impact of sales time on purchase amount.

This approach demonstrates the potential of LLMs in simplifying complex data analysis tasks, enabling analysts to focus on extracting valuable insights and making data-driven decisions. By using natural language to interact with the data, we streamlined the analysis process and gained a deeper understanding of the Columbia Plus Mart retail data. This approach can be applied to various datasets and industries, unlocking new possibilities for data exploration and analysis.